
leverage this great work by https://github.com/dohliam  from  
- original : https://github.com/kfcd/chaizi
- my fork: https://github.com/wgong/chaizi

In [1]:
from utils import *

## Bulk Update t_zi_part by leveraging ShuoWenJieZi and ChaiZi work

In [2]:
sql_stmt = """
with parts as (
	select * from t_zi_part where zi_left is not null and zi_left != '' 
    and zi_right is not null and is_active='Y' and zi_mid is null
	and zi_left not in ( '氵', '宀', '肉', '人', '水')  -- already done
) 
select zi_left,count(*) from parts group by zi_left 
order by count(*) desc;
"""

In [3]:
with DBConn() as _conn:
    df_count = pd.read_sql(sql_stmt, _conn)

In [4]:
df_count

,zi_left,count(*)
0,木,210
1,艸,203
2,手,138
3,心,136
4,口,110
...,...,...
289,。,1
290,纟,1
291,扌,1
292,己,1


In [5]:
df_count.to_csv("../devs/sql/parts_count-20240118.csv")

## Review ChaiZi work

In [3]:
file_caizi = "./data/chaizi-jt.txt"

In [14]:
caizi = {}
with open(file_caizi, encoding="utf-8") as f: 
    tmp = f.read()
    for i in tmp.split("\n"):
        x = i.strip()
        if not x: continue
        # t = ["".join(j.split()) for j in x.split("\t")]
        t = x.split("\t")
        caizi[t[0]] = " / ".join(t[1:])

In [15]:
caizi

{'丁': '一 亅',
 '丂': '一 乙',
 '七': '一 乚 / 丿 乚',
 '丄': '丨 一',
 '丅': '一 丨',
 '万': '一 勹 / 一 刀',
 '丈': '一 乂',
 '三': '一 二 / 一 一 一',
 '上': '一 丨 一 / 丶 丨 一',
 '下': '一 卜 / 一 丨 丶',
 '丌': '一 丿 丨',
 '不': '一 丿 丨 丶',
 '丐': '下 勹 / 正 丿',
 '丑': '㇆ 一 丨 一',
 '丒': '刃 一',
 '且': '月 一',
 '丕': '不 一',
 '世': '廿 丨 一',
 '丗': '廿 丨 / 十 凵',
 '丘': '厂 工 / 厂 丅 一',
 '丙': '一 人 冂',
 '丞': '氶 一',
 '丢': '丿 去',
 '丠': '北 一',
 '両': '一 山 冂',
 '并': '丷 开',
 '丫': '丶 丶 丨',
 '丬': '冫 丨',
 '中': '口 丨',
 '丮': '乁 二 丨',
 '丯': '丨 彡',
 '丰': '丿 二 丨 / 三 丨',
 '串': '口 口 丨 / 中 口',
 '丳': '串 丿',
 '丵': '业 丷 干',
 '丷': '丶 丿',
 '丸': '丶 九',
 '丹': '冂 丶 一',
 '主': '丶 王',
 '丼': '井 丶',
 '乂': '乀 丿',
 '乃': '丿 ㇆ ㇆',
 '久': '丿 入',
 '乆': '𠃊 人',
 '乇': '丿 七',
 '幺': '𠃊 厶',
 '乍': '丿 丅 二',
 '乎': '丿 丷 一 亅',
 '乏': '丿 之',
 '乑': '丿 丨 丿 丿 丶 乀',
 '乒': '丘 丿',
 '乓': '丘 丶',
 '乕': '厂 一 巿 / 厂 二 巾',
 '乖': '千 北',
 '乘': '禾 北 / 禾 丬 匕',
 '乜': '卩 乚',
 '九': '丿 乙',
 '乞': '丿 一 乙',
 '也': '卩 丨 乚',
 '乢': '山 乙 / 山 乚',
 '乣': '幺 乙 / 幺 乚',
 '乨': '台 乙 / 台 乚',
 '乩': '占 乙',
 '乪': '乙 田',
 '乫': '加 乙',
 

In [20]:
data = []
for k,v in caizi.items():
    data.append([k,v, '', 'Y'])

In [21]:
df = pd.DataFrame(data, columns=["zi","caizi","tags","is_active"])

In [22]:
df.shape 

(14563, 4)

In [23]:
df.head()

,zi,caizi,tags,is_active
0,丁,一 亅,,Y
1,丂,一 乙,,Y
2,七,一 乚 / 丿 乚,,Y
3,丄,丨 一,,Y
4,丅,一 丨,,Y


In [25]:
with DBConn() as _conn:
    df.to_sql("w_caizi", _conn, if_exists="replace")

The following are not yet decomposed

In [ ]:
sql_stmt = """
select c.zi, c.caizi, p.*
from t_zi_part p join w_caizi c on p.zi = c.zi
where 1=1
and p.zi_left is null and p.zi_left_down is null and p.zi_left_up is null 
and p.zi_right is null and p.zi_right_down is null and p.zi_right_up is null 
and p.zi_up is null and p.zi_mid is null and p.zi_down is null and p.zi_mid_in is null and p.zi_mid_out is null
;
-- 2449 are not yet decomposed
"""